### Back to Instance Properties

In [ ]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, onwer_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)

In [2]:
import weakref

In [13]:
class IntegerValue:
    def __init__(self):
        self.values = weakref.WeakKeyDictionary()

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, onwer_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)

In [14]:
class Point:
    x = IntegerValue()

In [22]:
p = Point()
print(hex(id(p)))

0x7fd146122ab0


In [23]:
p.x = 100.1

In [24]:
p.x

100

In [25]:
Point.x.values.keyrefs()

[<weakref at 0x7fd145bea3e0; to 'Point' at 0x7fd146122ab0>]

In [26]:
del p

In [27]:
Point.x.values.keyrefs()

[]

In [36]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = int(value)

    def __get__(self, instance, onwer_class):
        if instance is None:
            return self
        else:
            return self.values.get(id(instance))

In [37]:
class Point:
    x = IntegerValue()

    def __init__(self, x):
        self.x = x

    def __eq__(self, other):
        return isinstance(other, Point) and self.x == other.x

In [38]:
p = Point(10.1)

In [39]:
p.x

10

In [40]:
p.x = 20.2

In [43]:
id(p), Point.x.values

(140536800479952, {140536800479952: 20})

In [44]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [45]:
p_id = id(p)

In [46]:
ref_count(p_id)

1

In [47]:
del p

In [48]:
ref_count(p_id)

100

In [49]:
Point.x.values

{140536800479952: 20}

In [50]:
p = Point(10.1)

In [51]:
weak_p = weakref.ref(p)

In [52]:
ref_count(id(p))

1

In [53]:
del p

In [54]:
weak_p

<weakref at 0x7fd145948e50; dead>

In [55]:
def obj_destroyed(obj):
    print(f'{obj} is being destroyed')

In [57]:
p = Point(10.1)
w = weakref.ref(p, obj_destroyed)

In [58]:
del p

<weakref at 0x7fd145b52c50; dead> is being destroyed


In [59]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), int(value))

    def __get__(self, instance, onwer_class):
        if instance is None:
            return self
        else:
            # # value_tuple = self.values.get(id(instance))
            # value_tuple = self.values[id(instance)]
            # return value_tuple[1]
            # Or shortest:
            return self.values[id(instance)][1]

    def _remove_object(self, weak_ref):
        print(f'removing dead entry for {weak_ref}')



In [60]:
class Point:
    x = IntegerValue()

In [61]:
p1 = Point()
p2 = Point()

In [62]:
p1.x, p2.x = 10.1, 100.1

In [63]:
p1.x, p2.x

(10, 100)

In [64]:
ref_count(id(p1)), ref_count(id(p2))

(1, 1)

In [65]:
del p1

removing dead entry for <weakref at 0x7fd145b11f80; dead>


In [66]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), int(value))

    def __get__(self, instance, onwer_class):
        if instance is None:
            return self
        else:
            # # value_tuple = self.values.get(id(instance))
            # value_tuple = self.values[id(instance)]
            # return value_tuple[1]
            # Or shortest:
            return self.values[id(instance)][1]

    def _remove_object(self, weak_ref):
        # reverse_lookup = [key for key, value in self.values.items()
        #                   if value[0] is weak_ref]
        # if reverse_lookup:
        #     key = reverse_lookup[0]
        #     del self.value[key]

        # or using a for loop:
        for key, value in self.values.items():
            if value[0] is weak_ref:
                del self.values[key]
                break

In [67]:
class Point:
    x = IntegerValue()

In [68]:
p = Point()
p.x = 10.1

In [69]:
p.x

10

In [70]:
Point.x.values

{140536798698560: (<weakref at 0x7fd145bc2d90; to 'Point' at 0x7fd145f6dc40>,
  10)}

In [71]:
del p

In [72]:
Point.x.values

{}

In [73]:
class Person:
    pass

In [74]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [75]:
hasattr(Person.__weakref__, '__get__')

True

In [76]:
hasattr(Person.__weakref__, '__set__')

True

In [77]:
p = Person()

In [78]:
hasattr(p, '__weakref__')

True

In [79]:
print(p.__weakref__)

None


In [81]:
w = weakref.ref(p)

In [82]:
print(p.__weakref__)

<weakref at 0x7fd145b609a0; to 'Person' at 0x7fd146987380>


In [89]:
class Person:
    __slots__ = 'name',

In [90]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__slots__': ('name',),
              'name': <member 'name' of 'Person' objects>,
              '__doc__': None})

In [91]:
p = Person()

In [92]:
hasattr(p, '__weakref__')

False

In [93]:
w = weakref.ref(p)

TypeError: cannot create weak reference to 'Person' object

In [95]:
class Person:
    __slots__ = 'name', '__weakref__'

In [96]:
p = Person()

In [97]:
weakref.ref(p)

<weakref at 0x7fd145bfb650; to 'Person' at 0x7fd146a03490>

In [136]:
class ValidString:
    def __init__(self, min_length=0, max_length=255):
        self.data = {}
        self._min_length = min_length
        self._max_length = max_length

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError('Value must be a string')
        if len(value) < self._min_length:
            raise ValueError(
                f'Value should be at least {self._min_length} characters.'
            )
        if len(value) > self._max_length:
            raise ValueError(
                f'Value shoudl not exceed {self._max_length} characters.'
            )
        self.data[id(instance)] = (weakref.ref(instance, self._finalize_instance), value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.data.get(id(instance))
            return value_tuple[1]

    def _finalize_instance(self, weak_ref):
        reverse_lookup = [key for key, value in self.data.items()
                          if value[0] is weak_ref]
        if reverse_lookup:
            key = reverse_lookup[0]
            del self.data[key]

In [137]:
class Person:
    __slots__ = '__weakref__',

    first_name = ValidString(1, 100)
    last_name = ValidString(1, 100)

    def __eq__(self, other):
        return (
            isinstance(other, Person) and
            self.first_name == other.first_name and
            self.last_name == other.last_name
        )

class BankAccount:
    __slots__ = '__weakref__',
    account_number = ValidString(5, 255)

    def __eq__(self, other):
        return isinstance(other, BankAccount) and self.account_number == other.account_number

In [138]:
p1 = Person()

In [139]:
try:
    p1.first_name = ''
except ValueError as ex:
    print(ex)

Value should be at least 1 characters.


In [140]:
p2 = Person()

In [141]:
p1.first_name, p1.last_name = 'Guido', 'van Rossum'
p2.first_name, p2.last_name = 'Raymond', 'Hettinger'

In [142]:
b1, b2 = BankAccount(), BankAccount()

In [143]:
b1.account_number, b2.account_number = 'Savings', 'Checking'

In [144]:
p1.first_name, p1.last_name

('Guido', 'van Rossum')

In [145]:
p2.first_name, p2.last_name

('Raymond', 'Hettinger')

In [146]:
b1.account_number, b2.account_number

('Savings', 'Checking')

In [147]:
Person.first_name.data

{140536809732816: (<weakref at 0x7fd1457bd850; to 'Person' at 0x7fd1469f3ad0>,
  'Guido'),
 140536809728304: (<weakref at 0x7fd1457d90d0; to 'Person' at 0x7fd1469f2930>,
  'Raymond')}

In [148]:
Person.last_name.data

{140536809732816: (<weakref at 0x7fd145beb0b0; to 'Person' at 0x7fd1469f3ad0>,
  'van Rossum'),
 140536809728304: (<weakref at 0x7fd1457d8810; to 'Person' at 0x7fd1469f2930>,
  'Hettinger')}

In [149]:
BankAccount.account_number.data

{140536809721632: (<weakref at 0x7fd1458d06d0; to 'BankAccount' at 0x7fd1469f0f20>,
  'Savings'),
 140536809731616: (<weakref at 0x7fd1457c33d0; to 'BankAccount' at 0x7fd1469f3620>,
  'Checking')}

In [150]:
del p1
del p2
del b1
del b2

In [151]:
Person.first_name.data

{}